<a href="https://colab.research.google.com/github/skandvj/hw1p2/blob/main/HW1P2_F24_Starter_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 28 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# Libraries

In [ ]:
!pip install torchsummaryX==1.1.0 wandb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 13.0 MB/s eta 0:00:00


Install Required Libraries

In [ ]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
#''' If you are using colab, you can import google drive to save model checkpoints in a folder
    #If you want to use it, uncomment the two lines below
#'''
#from google.colab import drive
#drive.mount('/content/drive')

NotImplementedError: Mounting drive is unsupported in this environment. Use PyDrive instead. See examples at https://colab.research.google.com/notebooks/io.ipynb#scrollTo=7taylj9wpsA2.

In [ ]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [ ]:
#!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
#!mkdir /root/.kaggle

#with open("/root/.kaggle/kaggle.json", "w+") as f:
    #f.write('{"username":"skandv","key":"a290ff562f3888fb78632ca4cf208766"}')
    # Put your kaggle username & key here

#!chmod 600 /root/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73249 sha256=55e6a827d30ee6f50ea0191dd06b0038c9cc74ef0268a54b25dfab109865e104
  Stored in directory: /root/.cache/pip/wheels/0b/76/ca/e58f8afa83166a0e68f0d5cd2e7f99d260bdc40e35da080eee
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.6.14
    Uninstalling kaggle-1.6.14:
      Successfully uninstalled kaggle-1.6.14


In [ ]:
# commands to download data from kaggle
!kaggle competitions download -c 11785-hw1p2-f24

!unzip -qo /content/11785-hw1p2-f24.zip -d '/content'

100% 3.97G/3.98G [00:15<00:00, 302MB/s]
100% 3.98G/3.98G [00:15<00:00, 278MB/s]


In [ ]:
!pip install chardet
import chardet

# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [ ]:
# Dataset class to load train and validation data



class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes = PHONEMES, context=0, partition= "train-clean-100"): # Feel free to add more arguments


        self.context    = context
        self.phonemes   = phonemes


        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir       = os.path.join(root, partition, 'mfcc')
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data using root
        self.transcript_dir = os.path.join(root, partition, 'transcript')

        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        mfcc_names          = sorted(os.listdir(self.mfcc_dir))
        # TODO: List files in self.transcript_dir using os.listdir in sorted order
        transcript_names    = sorted(os.listdir(self.transcript_dir))

        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []

        # TODO: Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
        #   Load a single mfcc
            mfcc_path        = os.path.join(self.mfcc_dir, mfcc_names[i])
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc = np.load(mfcc_path)
            mean = np.mean(mfcc, axis=0)
            n_mfcc=mfcc-mean
            std =np.std(mfcc, axis=0)
            n_mfcc=n_mfcc/std
            mfcc=n_mfcc


        #   Load the corresponding transcript
            transcript_path  = os.path.join(self.transcript_dir, mfcc_names[i]) # Remove [SOS] and [EOS] from the transcript
            # (Is there an efficient way to do this without traversing through the transcript?)
            transcript = np.load(transcript_path)
            transcript = transcript[1:-1]
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        # NOTE:
        # Each mfcc is of shape T1 x 28, T2 x 28, ...
        # Each transcript is of shape (T1+2), (T2+2) before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that
        # the final shape is T x 28 (Where T = T1 + T2 + ...)
        self.mfccs          = np.concatenate(self.mfccs, axis=0)

        # TODO: Concatenate all transcripts in self.transcripts such that
        # the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts    = np.concatenate(self.transcripts, axis=0)
        # Hint: Use numpy to concatenate

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # Take some time to think about what we have done.
        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        zero_padding = np.zeros((self.context, self.mfccs.shape[1]))
        self.mfccs = np.vstack((zero_padding, self.mfccs, zero_padding))


        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such.
        # Hence, we map these phonemes to integers
        phoneme_to_indx ={phoneme: idx for idx, phoneme in enumerate(self.phonemes)}
        self.transcripts = np.array([phoneme_to_indx[phoneme] for phoneme in self.transcripts])
        # TODO: Map the phonemes to their corresponding list indexes in self.phonemes

        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):

        # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        frames = self.mfccs[ind:ind + 2 * self.context + 1]
        # After slicing, you get an array of shape 2*context+1 x 28. But our MLP needs 1d data and not 2d.
        frames = frames.flatten() # TODO: Flatten to get 1d data

        frames      = torch.FloatTensor(frames) # Convert to tensors
        phonemes    = torch.tensor(self.transcripts[ind])

        return frames, phonemes

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):
    #pass
    def __init__(self, root, context=0, partition="test-clean"):  # Partition points to test data
        self.context = context

        # Set MFCC directory based on the test partition
        self.mfcc_dir = os.path.join(root, partition, 'mfcc')  # E.g., /content/data/test-clean/mfcc

        # List the files in sorted order
        mfcc_names = sorted(os.listdir(self.mfcc_dir))

        self.mfccs = []

        # Load and preprocess MFCCs
        for mfcc_name in mfcc_names:
            # Load MFCC
            mfcc_path = os.path.join(self.mfcc_dir, mfcc_name)
            mfcc = np.load(mfcc_path)  # Assuming .npy format for MFCC
            # Cepstral Normalization (mean-variance normalization)
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)

            # Append the processed data
            self.mfccs.append(mfcc)

        # Concatenate all MFCCs
        self.mfccs = np.concatenate(self.mfccs, axis=0)

        self.length = len(self.mfccs)

        # Padding zeros for context frames
        self.mfccs = np.pad(self.mfccs, ((context, context), (0, 0)), mode='constant', constant_values=0)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        # Extract frames with context
        frames = self.mfccs[ind:ind + 2 * self.context + 1]

        # Flatten the frames for MLP input
        frames = frames.flatten()

        frames = torch.FloatTensor(frames)  # Convert to tensor
        #phonemes = torch.tensor(self.transcripts[ind]) # Access transcripts using filename key

        return frames
    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments.

In [ ]:
config = {
    'epochs'        : 20,
    'batch_size'    : 2048,
    'context'       : 35,
    'init_lr'       : 1e-3,
    'weight_decay'  : 1e-3,
    'architecture'  : 'medium-cutoff',
    'dropout'       : 0.2

    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters

}

# Create Datasets

In [ ]:
!ls /content/11785-f24-hw1p2

dev-clean  test-clean  train-clean-100


In [ ]:
root_path = '/content/11785-f24-hw1p2'
#TODO: Create a dataset object using the AudioDataset class for the training data

train_data = AudioDataset(root=root_path, context=config['context'], partition="train-clean-100")

# TODO: Create a dataset object using the AudioDataset class for the validation data
val_data = AudioDataset(root=root_path, context=config['context'], partition="dev-clean")

# TODO: Create a dataset object using the AudioTestDataset class for the test data
test_data = AudioTestDataset(root=root_path, context=config['context'], partition="test-clean")

# Print dataset statistics
print(f"Training dataset size: {len(train_data)} samples")
print(f"Validation dataset size: {len(val_data)} samples")
print(f"Test dataset size: {len(test_data)} samples")

# Example of retrieving a sample
sample_idx = 0
train_sample = train_data[sample_idx]
print(f"Training sample {sample_idx} MFCC shape: {train_sample[0].shape}")
print(f"Training sample {sample_idx} Phoneme: {train_sample[1]}")

Training dataset size: 36091157 samples
Validation dataset size: 1928204 samples
Test dataset size: 1934138 samples
Training sample 0 MFCC shape: torch.Size([1988])
Training sample 0 Phoneme: 0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2*config['context']+1)*28)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  2048
Context        :  35
Input size     :  1988
Output symbols :  42
Train dataset samples = 36091157, batches = 17623
Validation dataset samples = 1928204, batches = 942
Test dataset samples = 1934138, batches = 945


In [ ]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([2048, 1988]) torch.Size([2048])


# Network Architecture


Started with 2 Linear Layers with RELU Activation function, the accuracy was not increasing beyond ~76% even with 5 and 7 layers.

After Replacing RELU with GELU, Performance of the model increased.

Final Architecture

Layer 1: Linear layer with 2048 units
Batch Normalization
GELU activation
Dropout (0.3)

Layer 2: Linear layer with 2048 units
Batch Normalization
GELU activation
Dropout (0.4)

Layer 3:Linear layer with 2048 units
Batch Normalization
GELU activation
Dropout (0.3)

Layer 4:Linear layer with 2048 units
Batch Normalization
GELU activation
Dropout (0.4)

Layer 5:Linear layer with 1024 units
Batch Normalization
GELU activation
Dropout (0.2)

Layer 6:Linear layer with 512 units
Batch Normalization
GELU activation
Dropout (0.2)

Layer 7:Linear layer with 256 units
Batch Normalization
GELU activation
Dropout (0.1)
Output Layer: Linear layer mapping to the number of phoneme classes.

In [ ]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):

        super(Network, self).__init__()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 2048),  # Reduce layer size
    torch.nn.BatchNorm1d(2048),
    torch.nn.GELU(),
    torch.nn.Dropout(p=0.3),

    torch.nn.Linear(2048, 2048),  # Reduce layer size
    torch.nn.BatchNorm1d(2048),
    torch.nn.GELU(),
    torch.nn.Dropout(p=0.4),

    torch.nn.Linear(2048, 2048),  # Reduce layer size
    torch.nn.BatchNorm1d(2048),
    torch.nn.GELU(),
    torch.nn.Dropout(p=0.3),

    torch.nn.Linear(2048, 2048),  # Reduce layer size
    torch.nn.BatchNorm1d(2048),
    torch.nn.GELU(),
    torch.nn.Dropout(p=0.4),

    torch.nn.Linear(2048, 1024),  # Reduce layer size
    torch.nn.BatchNorm1d(1024),
    torch.nn.GELU(),
    torch.nn.Dropout(p=0.2),

    torch.nn.Linear(1024, 512),         # Reduce layer size
    torch.nn.BatchNorm1d(512),
    torch.nn.GELU(),
    torch.nn.Dropout(p=0.2),

    torch.nn.Linear(512,256),         # Reduce layer size
    torch.nn.BatchNorm1d(256),
    torch.nn.GELU(),
    torch.nn.Dropout(p=0.1),

    torch.nn.Linear(256, output_size)
        )

    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler.

In [ ]:
INPUT_SIZE  = (2*config['context'] + 1) * 28 # Why is this the case?
model       = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
summary(model, frames.to(device))
# Check number of parameters of your network
# Remember, you are limited to 20 million parameters for HW1 (including ensembles)

----------------------------------------------------------------------------------------------------
Layer                   Kernel Shape         Output Shape         # Params (K)      # Mult-Adds (M)
0_Linear                [1988, 2048]         [2048, 2048]             4,073.47                 4.07
1_BatchNorm1d                 [2048]         [2048, 2048]                 4.10                 0.00
2_GELU                             -         [2048, 2048]                    -                    -
3_Dropout                          -         [2048, 2048]                    -                    -
4_Linear                [2048, 2048]         [2048, 2048]             4,196.35                 4.19
5_BatchNorm1d                 [2048]         [2048, 2048]                 4.10                 0.00
6_GELU                             -         [2048, 2048]                    -                    -
7_Dropout                          -         [2048, 2048]                    -                    -

Tried Different optimizers for optimizing the model during training, several optimizers and learning rate schedulers were tested to enhance performance and reduce training time.

Loss Function
We used CrossEntropyLoss because the task is a multi-class classification problem where each frame is assigned one phoneme label

Optimizer
The primary optimizer tested was AdamW, a variant of the Adam optimizer that includes weight decay for regularization. This optimizer was chosen due to its effectiveness in handling complex, high-dimensional models like the deep neural network used here. The learning rate was initialized according to the config['init_lr']

Learning Rate Schedulers:
To dynamically adjust the learning rate during training, we tested several schedulers:

CosineAnnealingWarmRestarts (Found Best)
StepLR
CosineAnnealing




In [ ]:
criterion = torch.nn.CrossEntropyLoss() # Defining Loss function.
# We use CE because the task is multi-class classification

optimizer = torch.optim.AdamW(model.parameters(), lr= config['init_lr'],weight_decay=1e-4) #Defining Optimizer
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0= 3, T_mult=2 ,eta_min = 1e-4)  # Add this line after optimizer
# Recommended : Define Scheduler for Learning Rate,
# including but not limited to StepLR, MultiStep, CosineAnnealing, CosineAnnealingWithWarmRestarts, ReduceLROnPlateau, etc.
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html


In [ ]:
from torch.optim.lr_scheduler import StepLR

#scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)



# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

891

In [ ]:
def train(model, dataloader, optimizer, criterion):

    model.train()
    tloss, tacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Move Data to Device (Ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        ### Forward Propagation
        logits  = model(frames)

        ### Loss Calculation
        loss    = criterion(logits, phonemes)

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()

        tloss   += loss.item()
        predictions = torch.argmax(logits, dim=1)
        tacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss   /= len(train_loader)
    tacc    /= len(train_loader)

    return tloss, tacc

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode
    vloss, vacc = 0, 0 # Monitoring loss and accuracy
    batch_bar   = tqdm(total=len(dataloader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        ### Move data to device (ideally GPU)
        frames      = frames.to(device)
        phonemes    = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            ### Forward Propagation
            logits  = model(frames)
            ### Loss Calculation
            loss    = criterion(logits, phonemes)

        vloss   += loss.item()
        predictions = torch.argmax(logits, dim=1)
        vacc    += torch.sum(torch.argmax(logits, dim= 1) == phonemes).item()/logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))
        batch_bar.update()

        ### Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss   /= len(val_loader)
    vacc    /= len(val_loader)

    return vloss, vacc

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb.

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [ ]:
wandb.login(key="f9cf09dac50889564c1f8fb34fff472412382ff7") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name    = "first-run", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account
    config  = config ### Wandb Config for your run
)

In [ ]:
### Save your model architecture as a string with str(model)
model_arch  = str(model)

### Save it in a txt file
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20240920_102254-a4zuau13/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")
config = {'epochs': 40,
          'save_best_only': True  # Save the best model based on validation accuracy
}
best_val_acc = 0

for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})
        # Save the model if validation accuracy improves
    if config.get('save_best_only', False):
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            # Save the best model
            torch.save(model.state_dict(), "best_model.pth")
            wandb.save("best_model.pth")
      # Clear memory
    torch.cuda.empty_cache()
    gc.collect()

# Finalize wandb run
wandb.finish()

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best



Epoch 1/40


Train:   0%|          | 0/17623 [00:00<?, ?it/s]

# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *sample_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval()# TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.no_grad(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):

            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes = torch.argmax(logits, dim=1)

            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            # TODO
            test_predictions.extend(predicted_phonemes.cpu().numpy())


    return test_predictions

In [ ]:
predictions = test(model, test_loader)

  0%|          | 0/945 [00:00<?, ?it/s]

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
      phoneme = PHONEMES[predictions[i]]
      f.write("{},{}\n".format(i, phoneme))

In [ ]:
### Finish your wandb run
run.finish()

lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▂▂▃▃▄▄▅▅▆▆▇▇▇███
train_loss,█▇▇▆▆▅▅▅▄▄▃▃▂▂▂▁▁
val_acc,▁▂▂▃▃▂▄▅▄▆▅▆▇▇▇▇█
valid_loss,█▇█▇▆▆▅▄▅▃▄▃▃▃▃▃▁
lr,0.001
train_acc,85.7714
train_loss,0.41001
val_acc,86.02711
valid_loss,0.41421


In [ ]:
### Submit to kaggle competition using kaggle API (Uncomment below to use)
#!kaggle competitions submit -c 11785-hw1p2-f24 -f ./submission.csv -m "Test Submission"

### However, its always safer to download the csv file and then upload to kaggle

100% 19.3M/19.3M [00:01<00:00, 19.5MB/s]
Successfully submitted to 11785 HW1P2 Fall 2024